# Import

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

from hnc.hnc.PIE import Plasma_of_Ions_and_Electrons as PLASMA

from hnc.hnc.constants import *
from hnc.hnc.misc import n_from_rs, rs_from_n

from pandas import read_csv

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
                
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}

</style>
""")


ModuleNotFoundError: No module named 'mendeleev'


# Function that sets up HNC QSP 

### Create and run plasma

In [3]:
name='Al'
Z, A = 13, 27
mi = m_p * A
ρ_gpercc =  2.7 
ni_cc = ρ_gpercc/(mi*AU_to_g)
ri_cm = rs_from_n(ni_cc)
ri_AU = ri_cm*cm_to_AU


### Make βu_ii, βu_fi from using bound density from AA
If we assume the bound state does not deform at all, then we can calculate potentials based on it easily via integration

### Solving

In [25]:
from scipy.optimize import least_squares
from scipy.interpolate import interp1d

def custom_flat(hnc_matrix):
    hnc_flat = np.concatenate([hnc_matrix[0,0],hnc_matrix[0,1],hnc_matrix[1,1]])
    return hnc_flat

def custom_unflat(hnc_flat):
    hnc00, hnc01, hnc11 = np.split(hnc_flat,3)
    hnc_matrix = np.zeros((2,2,len(hnc00)))
    hnc_matrix[0,0] = hnc00
    hnc_matrix[1,0] = hnc01
    hnc_matrix[0,1] = hnc01
    hnc_matrix[1,1] = hnc11
    return hnc_matrix

def solve_least_squares(plasma, sparse_k_array, c_s_k_guess=None, ftol=1e-8):
    if c_s_k_guess is None:
        c_s_k_guess = plasma.hnc.c_s_k_matrix

    # Create functions for converting between sparse, not sparse and flattened versions of c_s_k
    get_sparse_csk = lambda c_s_k_flat: np.concatenate([interp1d(plasma.hnc.k_array, c_s_k)(sparse_k_array) for c_s_k in np.split(c_s_k_flat,3) ])
    get_csk = lambda sparse_hnc_csk: interp1d(sparse_k_array, sparse_hnc_csk, kind='cubic')(plasma.hnc.k_array)
    get_csk_matrix_from_sparse_csk_flat =  lambda sparse_c_s_k_flat: custom_unflat(np.concatenate([get_csk(sparse_csk) for sparse_csk in np.split(sparse_c_s_k_flat,3)]))
    
    def min_function(sparse_c_s_k_flat):
        c_s_k_matrix = get_csk_matrix_from_sparse_csk_flat(sparse_c_s_k_flat)
        new_c_s_k_matrix = plasma.hnc.guess_c_s_k_matrix(c_s_k_matrix)
        plasma.hnc.c_s_k_matrix = new_c_s_k_matrix.copy()
        new_c_s_k_flat = custom_flat(new_c_s_k_matrix)
        return sparse_c_s_k_flat - get_sparse_csk(new_c_s_k_flat)
    
    c_s_k_flat_guess = custom_flat(c_s_k_guess)
    c_s_k_sparse_guess = get_sparse_csk(c_s_k_flat_guess)
    sol = least_squares(min_function, c_s_k_sparse_guess, verbose=2, ftol=ftol)#, bounds = c_s_k_bounds)
    plasma.hnc.c_s_k_matrix = get_csk_matrix_from_sparse_csk_flat(sol.x)
    plasma.hnc.set_all_matrices_from_csk(plasma.hnc.c_s_k_matrix)
    print(sol)

def best_shift(plasma, c_s_k_guess = None, ftol=1e-8, set=True):
    if c_s_k_guess is None:
        c_s_k_guess = plasma.hnc.c_s_k_matrix

    def min_function(multiplier_list):
        xii, xei, xee = multiplier_list
        multiplier_matrix = np.array([[xii,xei],[xei,xee]])
        c_s_k_matrix = multiplier_matrix[:,:,np.newaxis] * c_s_k_guess
        new_c_s_k_matrix = plasma.hnc.guess_c_s_k_matrix(c_s_k_matrix)
        
        return c_s_k_matrix.flatten() - new_c_s_k_matrix.flatten()
    
    sol = least_squares(min_function, np.ones((3)) , verbose=2, ftol=ftol)#, bounds = c_s_k_bounds)
    xii, xei, xee = sol.x
    multiplier_matrix = np.array([[xii,xei],[xei,xee]])
    
    plasma.hnc.c_s_k_matrix = multiplier_matrix[:,:,np.newaxis] * c_s_k_guess
    print(sol)
    return sol

In [26]:
def make_Ashcroft_βvei(atom, α=1, Rc = 0.4):
    # Assumes Rc in terms of Wigner-Seitz radius already as are hnc units
    k = atom.hnc.k_array
    βvei= atom.hnc.FT_k_2_r(  4*π*atom.qsp.Γei* (np.cos(k*Rc)*(1-α) + α*np.sin(k*Rc)/(k*Rc)) * (1/k**2 - 1/(k**2 + 1/atom.qsp.Λei**2)) )
    atom.hnc.βu_r_matrix[0,1] = βvei
    atom.hnc.βu_r_matrix[1,0] = βvei
    atom.hnc.set_βu_matrix(atom.hnc.βu_r_matrix)

In [48]:
def plasma_f(T_eV, Rc, Zbar, closure='svt'):
    plasma = PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
             βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
             qsp_options = {'Te_c_type':'DMC'},
             hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':closure},
             hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':2000},
             root_options={'options':{'maxfev':100,'xtol':1e-14}})
    
    # Make the smoothed Ashcroft potential 
    make_Ashcroft_βvei(plasma, α=0, Rc = Rc)
    
    return plasma

In [49]:
def ocp_f(T_eV, Zbar):
    ocp = PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
             βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
                 hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':'standard'},
                 hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':2000},
                 root_options={'options':{'maxfev':100,'xtol':1e-14}})
    ocp.run_ocp_hnc()
    return ocp

def yp_f(T_eV, Zbar):
    yukawa_plasma =  PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
                 βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
                 hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':'standard'},
                 hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':2000},
                 root_options={'options':{'maxfev':100,'xtol':1e-14}})
    yukawa_plasma.run_yukawa_hnc();
    return yukawa_plasma

In [44]:
def save_plasma(plasma, Rc):
    T_eV = plasma.Ti_eV
    Zbar = plasma.qsp.Zbar
    ocp = ocp_f( T_eV, Zbar)
    yp  = yp_f(T_eV, Zbar )
    
    # Make header
    info_header = (f"# Zach Johnson Dec/11/2023 CHNC for TCCW Paper" + '\n' + 
                   f"# Ti={T_eV:0.3e}[eV], Tie={plasma.qsp.Tie_c*AU_to_eV:0.3e}[eV] , Te_c={plasma.qsp.Te_c*AU_to_eV:0.3e}[eV], n_i={plasma.ni_cc:0.3e}[1/cc], n_e={Zbar*plasma.ni_cc:0.3e}[1/cc]" + '\n' + 
                   f"# ri={ri_AU:0.3f}, Zbar={Zbar:0.3f}, Rc={Rc:0.3f}[AU], Γii={plasma.qsp.Γii:0.3e}, Γei={plasma.qsp.Γei:0.3e} , Γee={plasma.qsp.Γee:0.3e}, κ={plasma.qsp.get_κ():0.3e}" 
                  )
    column_header = f"{'r/ri':10}  {'g_ii':10}  {'g_ei':10}  {'g_ee':10}  {'OCP:g_ii':10}  {'Yuk:g_ii':10}  {'βu_ii_eff':10}  {'βu_OCP':10}  {'βu_Yuk':10}"
    header = info_header + '\n' + column_header
    
    # Make save array
    save_array = np.array([  plasma.hnc.r_array, plasma.hnc.h_r_matrix[0,0]+1, plasma.hnc.h_r_matrix[0,1]+1, plasma.hnc.h_r_matrix[1,1]+1,
                           ocp.ocp_hnc.h_r_matrix[0,0]+1, yp.yuk_hnc.h_r_matrix[0,0] + 1, plasma.hnc.βueff_r_matrix[0,0],
                           ocp.ocp_hnc.βu_r_matrix[0,0], yp.yuk_hnc.βu_r_matrix[0,0]]).T
    
    np.savetxt(f"/home/zach/plasma/hnc/TCCW/Final_TCCW/results/CHNC_Al_rs{ri_AU:0.3f}_TeV{plasma.Ti_eV:0.3f}.dat", save_array, 
               header=header, fmt='%10.5e', comments='')

### Now Load Data from George
Rc is interpreted as the average bound radius, in AU in the file. Empty-core Ashcroft assumed.

In [45]:
GP_data   = read_csv("/home/zach/plasma/hnc/TCCW/Final_TCCW/GP_data/Al_combined.dat", header=0, delim_whitespace=True)
T_eV_list = GP_data['Te(eV)']
Rc_list   = GP_data['Rc'] # AU
Zbar_list = GP_data['Z_bar'] #Average atom Zbar GP_data
GP_data

,Te(eV),rho/rho0,Ne_b,Ne_quasi,Ne_sc,Z_bar,Rc,mu,nu_mom,nu_tot,sigma,k_th,L,G
0,0.1,1.0,10.000,0.0,3.000,3.000,0.514,0.288900,0.01607,0.09550,1.0740,0.01299,3.293,1.713000e-08
1,0.2,1.0,10.000,0.0,3.000,3.000,0.517,0.329200,0.01455,0.07732,1.0620,0.02592,3.321,2.510000e-08
2,0.5,1.0,10.000,0.0,3.000,3.000,0.518,0.330800,0.01410,0.07276,1.1130,0.06950,3.399,2.533000e-08
3,1.0,1.0,10.000,0.0,3.000,3.000,0.521,0.331400,0.01352,0.06448,1.2280,0.14220,3.153,2.608000e-08
4,2.0,1.0,10.000,0.0,3.000,3.000,0.525,0.327300,0.01493,0.05904,1.2480,0.22160,2.417,3.186000e-08
5,5.0,1.0,10.000,0.0,3.005,3.005,0.533,0.258500,0.02399,0.05507,1.0810,0.29680,1.494,6.148000e-08
6,10.0,1.0,9.932,0.0,3.068,3.068,0.538,0.007439,0.04736,0.07723,0.7355,0.35850,1.327,1.238000e-07
7,20.0,1.0,9.108,0.0,3.892,3.892,0.513,-0.692100,0.10870,0.20160,0.2711,0.51860,2.602,2.779000e-07
8,50.0,1.0,6.893,0.0,6.107,6.107,0.486,-3.522000,0.27120,0.55740,0.2203,1.42500,3.520,6.410000e-07
9,100.0,1.0,4.599,0.0,8.401,8.401,0.397,-9.565000,0.27220,0.59680,0.4190,5.61800,3.649,8.146000e-07


In [46]:
plasma_list = []

for T_eV, Rc, Zbar in zip(T_eV_list, Rc_list, Zbar_list):
    plasma = plasma_f(T_eV, Rc/ri_AU, Zbar )    
    plasma_list.append(plasma)

Te_in_eV: 0.100
Ti_in_eV: 0.100
Te_in_eV: 0.200
Ti_in_eV: 0.200
Te_in_eV: 0.500
Ti_in_eV: 0.500
Te_in_eV: 1.000
Ti_in_eV: 1.000
Te_in_eV: 2.000
Ti_in_eV: 2.000
Te_in_eV: 5.000
Ti_in_eV: 5.000
Te_in_eV: 10.000
Ti_in_eV: 10.000
Te_in_eV: 20.000
Ti_in_eV: 20.000
Te_in_eV: 50.000
Ti_in_eV: 50.000
Te_in_eV: 100.000
Ti_in_eV: 100.000
Te_in_eV: 200.000
Ti_in_eV: 200.000
Te_in_eV: 500.000
Ti_in_eV: 500.000
Te_in_eV: 1000.000
Ti_in_eV: 1000.000


In [56]:
guess_c_s_k_matrix = None
for Rc, plasma in zip(Rc_list[:4][::-1], plasma_list[:4][::-1]): 
    # if T_eV == 1.0:
    print(f"T_eV = {plasma.Ti_eV}")
    plasma.run_hnc(c_s_k_guess = guess_c_s_k_matrix)
    if plasma.hnc.tot_err_list[-1] > 1e-7:
        best_shift(plasma)
        solve_least_squares(plasma, np.linspace(plasma.hnc.k_array[0], plasma.hnc.k_array[-1], num=200))
        solve_least_squares(plasma, np.linspace(plasma.hnc.k_array[0], plasma.hnc.k_array[-1], num=400))
        save_plasma(plasma, Rc)
    else:
        save_plasma(plasma, Rc)
    # guess_c_s_k_matrix = plasma.hnc.c_s_k_matrix.copy()

T_eV = 1.0
0: Initial condition Total Error: 1.272e+01
QUIT: Last 10 iterations error has been increasing
Exiting status 2, reverting to best index so far: 501
Final iter:510, Total Error: 9.890e-04
Adjusting
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.1125e-03                                    9.96e+00    
       1              2         3.8753e-03      2.37e-04       8.78e-02       2.31e+00    
       2              3         3.6599e-03      2.15e-04       2.20e-02       1.48e+00    
       3              5         3.6279e-03      3.19e-05       5.49e-03       3.12e-01    
       4              6         3.6230e-03      4.96e-06       1.37e-03       3.70e-02    
       5              7         3.6178e-03      5.12e-06       3.43e-04       2.48e-02    
       6             16         3.6178e-03      0.00e+00       0.00e+00       2.48e-02    
`xtol` termination condition is satisfied.
Function evaluations

### Plot and Comparison to QMD

In [18]:
for plasma, Rc in zip(plasma_list[3:4], Rc_list[3:4]):    
    save_plasma(plasma, Rc)

Te_in_eV: 1.000
Ti_in_eV: 1.000
0: Initial condition Total Error: 2.552e+01
Starting Ng loop, using best index so far:  499
Exiting status 0, reverting to best index so far: 512
Final iter:515, Total Error: 9.836e-09
Te_in_eV: 1.000
Ti_in_eV: 1.000
0: Initial condition Total Error: 2.507e+02
Starting Ng loop, using best index so far:  499
Exiting status 0, reverting to best index so far: 499
Final iter:502, Total Error: 2.870e-09
